In [1]:
using Plots
using Statistics
using FFTW
using Random

In [105]:
function S(lattice::Array{Float64}, k::Float64)
    N = length(lattice)
    n = sum(lattice .== 0)
    s = 0.0
    for i in 1:N
        for j in 1:N
            if i != j && lattice[i] != 0 && lattice[j] != 0 
                s += exp(im * k * (i - j))
            end
        end
    end
    return abs(s)^2 / (N - n)
end;

In [97]:
function imperfect_lattice(N::Int, p::Float64)
    n = round(Int, N * p)
    lattice = ones(N)
    rand_indices = randperm(N)[1:n]
    lattice[rand_indices] .= 0
    return lattice
end;

In [82]:
function average_S(N::Int, p::Float64, k::Int, num_configs::Int)
    s_values = zeros(num_configs)
    for i in 1:num_configs
        lattice = imperfect_lattice(N, p)
        s_values[i] = S(lattice, k)
    end
    return mean(s_values)
end

In [201]:
function running_average_S(lattice::Array{Float64},k_arr::LinRange{Float64, Int64}, batchsize::Int)
    
    dk=k_arr[2]-k_arr[1]
    k_arr_extended=[LinRange(k[1]-batchsize*dk,k[1]-dk,batchsize-1);k_arr]
    S_arr=zeros(length(k_arr_extended))
    for (i,k) in enumerate(k_arr_extended)
        S_arr[i]=S(lattice,k)
    end
    s_mean_values = zeros(length(k_arr))
    for (i,k) in enumerate(k_arr)
        if i==1
            s_mean_values[i]=mean([S_arr[j] for j in i:(i+batchsize)])
        else
            s_mean_values[i]=(s_mean_values[i-1]*batchsize - S_arr[i-1] + S_arr[i+batchsize-1])/batchsize
        end
    end
    return s_mean_values
end;

In [216]:
function plot_averages(N::Int, p::Float64, k_arr::LinRange{Float64, Int64}, num_configs::Int,batchsize::Int)
    lattice = imperfect_lattice(N, p)
    avg_S = [average_S(N, p, k, num_configs) for k in k_arr]
    running_avg_S = running_average_S(lattice,k_arr,batchsize)
    #scatter(k_arr, running_avg_S, label="Running Average",yscale=:log10)
    #scatter!(k_arr,avg_S, label="Average S(k)",yscale=:log10)
    plot(k_arr, running_avg_S, label="Running Average",xlabel="k",ylabel="S(k)",yscale=:log10)
    plot!(k_arr,avg_S, label="Average S(k)",yscale=:log10)
    return running_avg_S
end;

In [226]:
N = 200
p = 0.5
nk=1001
batchsize=10
k_arr = LinRange(-π,π,nk)
dk=k_arr[2]-k_arr[1]
num_configs = 100
running_avg_S=plot_averages(N, p, k_arr, num_configs,batchsize);

In [227]:
height = maximum(running_avg_S)
width = length(running_avg_S[(running_avg_S.>=height/2) .& (running_avg_S.<=height)])*dk

0.06283185307179195

In [233]:
println("height = $height, width = $width ")

height = 325790.66100933996, width = 0.06283185307179195 


In [235]:
width/π

0.019999999999998755

In [237]:
π/50

0.06283185307179587

In [238]:
function running_avg_S_stats_for_N_p(N,p,batchsize,k_arr)
    dk=k_arr[2]-k_arr[1]
    lattice = imperfect_lattice(N, p)
    running_avg_S = running_average_S(lattice,k_arr,batchsize)
    height = maximum(running_avg_S)
    width = length(running_avg_S[(running_avg_S.>=height/2) .& (running_avg_S.<=height)])*dk
    return (height,width)
end;

In [ ]:
N_array=[10*m for m in 1:100]
height_width_arr = [running_avg_S_stats_for_N_p(Np,p,batchsize,k_arr) for Np in N_array];

In [ ]:
height=[ height_width[1] for height_width in height_width_arr]
width=[ height_width[2] for height_width in height_width_arr];

In [ ]:
plot(N_array,width,title="width(N)",xlabel="N",ylabel="width",xscale=:log10,yscale=:log10)

In [ ]:
plot(N_array,height,title="height(N)",xlabel="N",ylabel="height",xscale=:log10,yscale=:log10)